# データ内に存在しない親を探索

## 以降不足した親を繰り返し探す

In [22]:
#辞書データ
import pickle
# with open("data_extract/ontology_path.pickle", 'rb') as f:
# with open("data_extract/ontology_path_v2.pickle", 'rb') as f:
# with open("data_extract/ontology_path_v3.pickle", 'rb') as f:
# with open("data_extract/ontology_path_v4.pickle", 'rb') as f:
# with open("data_extract/ontology_path_v5.pickle", 'rb') as f:
with open("data_extract/ontology_path_v6.pickle", 'rb') as f:
    data_origin = pickle.load(f)

In [23]:
len(data_origin)

47336

In [2]:
# data_origin[0:3]

In [3]:
all_id = set()
all_p_taxon = set()
for d in data_origin:
    all_id.add(d["id"])
    all_p_taxon.add(d["parent_taxon"])

In [17]:
len(all_id)

47334

In [19]:
len(all_p_taxon)

9939

In [6]:
absent_p_taxon = all_p_taxon-all_id

In [7]:
len(absent_p_taxon)

3

In [8]:
absent_p_taxon

{None, 'Q160830', 'Q2043179'}

# 存在しない親を取得

In [9]:
# with open('sh3_absent_p_taxon.sh', 'w', encoding='UTF-8') as f:
# with open('sh4_absent_p_taxon.sh', 'w', encoding='UTF-8') as f:
# with open('sh5_absent_p_taxon.sh', 'w', encoding='UTF-8') as f:
with open('sh6_absent_p_taxon.sh', 'w', encoding='UTF-8') as f:
    f.write("#!/bin/bash\n")
    
    for item in absent_p_taxon:
        i = str(item)
#         command = "curl -sLH 'Accept: application/json' http://www.wikidata.org/entity/"+i+" | jq . > data_json_10/"+i+".json"
        command = "curl -sLH 'Accept: application/json' http://www.wikidata.org/entity/"+i+" | jq . > data_json_13/"+i+".json"
        f.write(command+"\n")

In [10]:
# !mkdir data_json_10

In [166]:
# !bash sh3_absent_p_taxon.sh
# !bash sh4_absent_p_taxon.sh
# !bash sh5_absent_p_taxon.sh #None.jsonができてしまったので，それは削除
!bash sh6_absent_p_taxon.sh #None.jsonができてしまったので，それは削除

parse error: Invalid numeric literal at line 1, column 10


# 以降はこれまでと同じ流れでdictに

In [108]:
# from prog1

In [11]:
import os
import json
import pickle

def logmaker(file_dir,file,error):
    with open(file_dir+"/"+file+"_log.txt", 'w',encoding="utf-8") as log_f:
        log_f.write(error+"\n")

def Make_jsonpikle(file_dir):
    files = os.listdir(file_dir)
    files_1 = [file.replace(".json","") for file in files]
    all_bird_dict = []

    for i in range(len(files_1)):
        file = files_1[i]
        print('\r%d / %d' %(i, len(files_1)), end='')

        try:
            with open(file_dir+'/'+file+".json", 'r',encoding="utf-8") as rf:
                json_dict = json.load(rf)
        except:
            logmaker(file_dir,file,"read json")
            continue


        bird_dict = {}
        url = "https://www.wikidata.org/wiki/"+file
        bird_dict["id"] = file
        
        try:
            bird_dict["en_name"] = json_dict['entities'][file]["labels"]["en"]["value"]
        except:
            bird_dict["en_name"] = None
            logmaker(file_dir,file,"en_name")
        try:
            bird_dict["ja_name"] = json_dict['entities'][file]["labels"]["ja"]["value"]
        except:
            bird_dict["ja_name"] = None

        
        bird_dict["en_aliases"] = {}
        cnt1 = 0
        try:
            for d in json_dict['entities'][file]["aliases"]["en"]:
                bird_dict["en_aliases"][cnt1] = d["value"]
                cnt1 = cnt1 + 1
        except:
            pass

        bird_dict["ja_aliases"] = {}
        cnt2 = 0
        try:
            for d in json_dict['entities'][file]["aliases"]["ja"]:
                bird_dict["ja_aliases"][cnt2]=d["value"]
                cnt2 = cnt2 + 1
        except:
            pass


        bird_dict["img_urls"] = {}
        cnt3 = 0
        try:
            for img in json_dict['entities'][file]["claims"]["P18"]:
                img_url = url+"#/media/File:"+img["mainsnak"]["datavalue"]["value"].replace(" ","%20")
                bird_dict["img_urls"][cnt3]=img_url
                cnt3 = cnt3 + 1
        except:
            pass

        try:
            bird_dict["taxon_rank"] = json_dict['entities'][file]["claims"]["P105"][0]["mainsnak"]["datavalue"]["value"]["id"]
        except:
            bird_dict["taxon_rank"] = None
            logmaker(file_dir,file,"taxon_rank")

        try:
            bird_dict["taxon_name"] = json_dict['entities'][file]["claims"]["P225"][0]["mainsnak"]["datavalue"]["value"]
        except:
            bird_dict["taxon_name"] = None
            logmaker(file_dir,file,"taxon_name")

        try:
            bird_dict["parent_taxon"] = json_dict['entities'][file]["claims"]["P171"][0]["mainsnak"]["datavalue"]["value"]["id"]
        except:
            bird_dict["parent_taxon"] = None
            logmaker(file_dir,file,"parent_taxon")
        all_bird_dict.append(bird_dict)


    with open("data_extract/"+file_dir+".pickle", 'wb') as bf:
        pickle.dump(all_bird_dict,bf)

In [13]:
# for i in [10]:
# for i in [11]:
# for i in [12]:
# for i in [13]:
#     Make_jsonpikle("data_json_"+str(i))

In [14]:
# with open("data_extract/data_json_10.pickle","rb") as f:
# with open("data_extract/data_json_11.pickle","rb") as f:
# with open("data_extract/data_json_12.pickle","rb") as f:
with open("data_extract/data_json_13.pickle","rb") as f:
    data_add = pickle.load(f)

In [15]:
data = data_add+data_origin

In [16]:
len(data)

47338

In [192]:
data_d = {}
for d in data:
    data_d[d["id"]]=d
    
ID_list = []
for v in data:
    ID_list.append(v["taxon_rank"])
    ID_list.append(v["parent_taxon"])
ID_set = set(ID_list)

ID_d = dict()
nodata_set = set()
for k in ID_set:
    if k in data_d:
        ID_d[k] = data_d[k]["en_name"]
    else:
        nodata_set.add(k)

ID_d_ja = dict()
nodata_set_ja = set()
for k in ID_set:
    if k in data_d:
        ID_d_ja[k] = data_d[k]["ja_name"]
    else:
        nodata_set_ja.add(k)

In [193]:
# data_origin

In [194]:
non_data = []
for d in data:
    try:
        d["taxon_rank_name"] = ID_d[d["taxon_rank"]]
        d["taxon_rank_ja_name"] = ID_d_ja[d["taxon_rank"]]

        d["parent_taxon_name"] = ID_d[d["parent_taxon"]]
        d["parent_taxon_ja_name"] = ID_d_ja[d["parent_taxon"]]

        s = ""
        for k,v in d["en_aliases"].items():
            if s == "":
                s = s + v
            else:
                s = s + "," +v
        d["en_aliases"] = s

        s = ""
        for k,v in d["ja_aliases"].items():
            if s == "":
                s = s + v
            else:
                s = s + "," +v
        d["ja_aliases"] = s

        s = ""
        for k,v in d["img_urls"].items():
            if s == "":
                s = s + v
            else:
                s = s + " " +v
        d["img_urls"] = s
    except:
        non_data.append(d["taxon_rank"])

In [195]:
from anytree import Node, RenderTree
node_set = set()

for d in data:
    _id = d["id"].replace("]","")
    if type(d["parent_taxon"]) == str:
        _parent = d["parent_taxon"].replace("]","")
    
    if d["parent_taxon"] in node_set:
        exec('%s = Node(d["id"],parent= %s)' % (_id,_parent))
        node_set.add(d["id"])
    elif d["parent_taxon"] != None:
        exec('%s = Node(d["parent_taxon"])' % (_parent))
        node_set.add(d["parent_taxon"])
        exec('%s = Node(d["id"],parent= %s)' % (_id,_parent))
        node_set.add(d["id"])
    else:
        exec('%s = Node(d["id"],parent= None)' % (_id))

In [196]:
for d in data:
    _id = d["id"].replace("]","")
    exec('d["path"] = str(%s).replace("Node(","").replace(")","")' % (_id))

In [197]:
dict_data = {}
for d in data:
    dict_data[d["id"]]=d

In [198]:
for d in data:
    try:
#         print(d)
        path_list = d["path"].replace("'/","").replace("'","").split("/")
        path_list

        ja_path_name = []
        en_path_name = []
        path_taxon_rank_name = []

        for path in path_list:
            if path in dict_data:
                if dict_data[path]["ja_name"] != "" and type(dict_data[path]["ja_name"])==str:
                    ja_path_name.append(dict_data[path]["ja_name"])
                    en_path_name.append(dict_data[path]["en_name"])
                else:
                    ja_path_name.append(dict_data[path]["en_name"])
                    en_path_name.append(dict_data[path]["en_name"])
                if dict_data[path]["taxon_rank_name"] != "" and type(dict_data[path]["ja_name"])==str:
                    path_taxon_rank_name.append(dict_data[path]["taxon_rank_name"])
                else:
                    path_taxon_rank_name.append("")
            else:
                ja_path_name.append(path)
                path_taxon_rank_name.append("Unknown")

        d["path"] = "/".join(path_list)
        d["ja_path_name"] = "/".join(ja_path_name)
        d["en_path_name"] = "/".join(en_path_name)
        d["path_taxon_rank_name"] = "/".join(path_taxon_rank_name)
    except:
        pass

In [199]:
data[0:3]

[{'id': 'Q23809240',
  'en_name': 'Dipnotetrapodomorpha',
  'ja_name': None,
  'en_aliases': {},
  'ja_aliases': {},
  'img_urls': {0: 'https://www.wikidata.org/wiki/Q23809240#/media/File:Dipnotetrapodomorpha.jpg'},
  'taxon_rank': None,
  'taxon_name': 'Dipnotetrapodomorpha',
  'parent_taxon': 'Q160830',
  'path': "'/Q160830/Q23809240'"},
 {'id': 'Q52762328',
  'en_name': 'Maniraptoromorpha',
  'ja_name': 'マニラプトル形類',
  'en_aliases': {},
  'ja_aliases': {},
  'img_urls': {0: 'https://www.wikidata.org/wiki/Q52762328#/media/File:Ornitholestes%20Royal%20Tyrrell%202.jpg'},
  'taxon_rank': None,
  'taxon_name': 'Maniraptoromorpha',
  'parent_taxon': 'Q2043179',
  'path': "'/Q2043179/Q52762328'"},
 {'id': 'Q1209254',
  'en_name': 'Tetrapodomorpha',
  'ja_name': None,
  'en_aliases': {0: 'Tetrapodomorph'},
  'ja_aliases': {},
  'img_urls': {0: 'https://www.wikidata.org/wiki/Q1209254#/media/File:Tiktaalik%20roseae%20(3705198718).jpg'},
  'taxon_rank': None,
  'taxon_name': 'Tetrapodomorpha',
 

In [200]:
filedname_list = ['id','en_name','ja_name','en_aliases','ja_aliases','img_urls','taxon_name','taxon_rank','taxon_rank_name','taxon_rank_ja_name','parent_taxon','parent_taxon_name','parent_taxon_ja_name',"path","ja_path_name","en_path_name","path_taxon_rank_name"]
import csv
# with open('data_extract/ontology_path_v2.tsv','w',encoding='utf-8') as csvfile:
# with open('data_extract/ontology_path_v3.tsv','w',encoding='utf-8') as csvfile:
# with open('data_extract/ontology_path_v4.tsv','w',encoding='utf-8') as csvfile:
# with open('data_extract/ontology_path_v5.tsv','w',encoding='utf-8') as csvfile:
with open('data_extract/ontology_path_v6.tsv','w',encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = filedname_list,delimiter = "\t")
    writer.writeheader()
    writer.writerows(data)
    
# with open("data_extract/ontology_path_v2.pickle", 'wb') as f:
# with open("data_extract/ontology_path_v3.pickle", 'wb') as f:
# with open("data_extract/ontology_path_v4.pickle", 'wb') as f:
# with open("data_extract/ontology_path_v5.pickle", 'wb') as f:
with open("data_extract/ontology_path_v6.pickle", 'wb') as f:
    pickle.dump(data,f)